In [69]:
import pandas as pd
import numpy as np
import urllib3
import urllib.request as re
import json

In [91]:
# Replace with your own file path to csv of parcels
FilePath = '/Users/hoperuse/summer2021internship/State Surplus/final_data.csv'
df = pd.read_csv(FilePath,index_col = False).drop(columns=['Unnamed: 0', 'index'])

# Filtering for parcels with some value in the numTransitStops column
df = df[df['numTransitStops'].notna()].reset_index(drop=True)

# Create appropriate columns for each score value, and filling each with initial null values
df['TransitScore'] = np.nan
df['WalkScore'] = np.nan
df['BikeScore'] = np.nan

In [ ]:
# Replace with your own WalkScore API key: https://www.walkscore.com/professional/api.php
api_key = '9c3fc53f0a33c604bd6a3e9c141ab7a6'
url = 'http://api.walkscore.com/score?format=json'

errors = 0
for i in range(len(df.index)):
   
    latitude = df.loc[i,'latitude']
    longitude = df.loc[i,'longitude']

    # If a given tuple doesn't have valid longitude / latitude coordinates, the API cannot calculate any scores, so set all to nan and 
    # continue to the next iteration
    if pd.isna(latitude) or pd.isna(longitude): 
        df.at[i, 'TransitScore'] = np.nan
        df.at[i,'WalkScore'] = np.nan
        df.at[i, 'BikeScore'] = np.nan
        continue
    try:
        request = url + '&lat=' + str(latitude) +'&lon=' + str(longitude) + '&transit=1&bike=1&wsapikey=' + api_key
        response = re.urlopen(request)
        data = json.load(response)

        # Using try statements to access each response from the API, as some locations don't return any of the values
        try:
            df.at[i, 'WalkScore'] = data['walkscore']
        except KeyError:
            df.at[i, 'WalkScore'] = np.nan
            
        try:
            df.at[i, 'TransitScore'] = data['transit']['score']
        except KeyError:
            df.at[i, 'TransitScore'] = np.nan

        try:
            df.at[i, 'BikeScore'] = data['bike']['score']
        except KeyError:
            df.at[i, 'BikeScore'] = np.nan
    
    except urllib3.exceptions.HTTPError as e:
        errors += 1
        print(e)

print('There were ' + str(errors) + ' errors...')

In [99]:
# load dataframe with transit information to CSV
df.to_csv('TransitScores.csv',index=False)